In [1]:
import pandas as pd
import script
from calitp_data_analysis.sql import to_snakecase

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

## Airtable

In [3]:
airtable = script.load_airtable(
    "./Transit Data Quality Issues-Open Issues by Distrcit (2).csv"
)

## Catastrophic

In [4]:
cat_df = script.load_catastrophic_errors(
    "./feed_info_expired_feeds_2023-10-02T14_54_58.563922Z.xlsx"
)

/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
cat_df

,service_name,district,uri,date
8,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-10-01
19,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-09-30
30,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-09-29
41,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-09-28
51,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-09-27
61,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-09-26
71,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-09-25
81,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-09-24
91,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-09-23
101,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-09-22


In [6]:
script.summarize_cat(cat_df, airtable)

,service_name,uri,# of days with expired feed
0,Merced The Bus,https://thebuslive.com/gtfs,3
1,Morro Bay Transit,https://mjcaction.com/MJC_GTFS_Public/morrobay_google_transit.zip,14
2,Plumas Transit Systems,https://data.trilliumtransit.com/gtfs/plumas-ca-us/plumas-ca-us.zip,14
3,TRACER,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,14


,service_name,uri,# of days with expired feed,gtfs_datasets,services,issue_type,description,airtable_ticket
0,Merced The Bus,https://thebuslive.com/gtfs,3,Merced VehiclePositions,Merced The Bus,GTFS Realtime Completeness Problem,Transit Joint Powers Authority for Merced County: GTFS-RT issue: complete outage of GTFS-RT Vehicle Positions Data,Yes
1,Morro Bay Transit,https://mjcaction.com/MJC_GTFS_Public/morrobay_google_transit.zip,14,Morro Bay Schedule,Morro Bay Transit,Expired Schedule Feed,City of Morro Bay: Follow-up about GTFS Schedule,Yes
2,Plumas Transit Systems,https://data.trilliumtransit.com/gtfs/plumas-ca-us/plumas-ca-us.zip,14,Plumas Schedule,Plumas Transit Systems,Expired Schedule Feed,"Based on the catastrophic errors dashboard, Plumas has not produced data since September 2. Per Evan, ""Plumas has a feed_info.txt file that says the feed was only valid until September 1 while the calendar says the feed is valid all the way until December 1. A strict interpretation of the GTFS spec says that any service beyond the feed end date is merely advisory, but some trip planners may still chose to display the service up until the end date."" This is the second ticket opened for Plumas.",Yes
3,TRACER,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,14,Tracy Schedule,TRACER,Expired Schedule Feed,City of Tracy: GTFS Schedule Feed Expired,Yes


## Incompleteness

In [7]:
script.load_api_511("API511.xlsx")

These are Bay Area feeds that keep track of RT


,new
13,AC TRANSIT
19,Bay Area Rapid Transit
22,Caltrain
34,County Connection
37,Dumbarton Express Consortium
40,Emery Go-Round
43,FAST
49,Golden Gate Transit
52,Livermore Amador Valley Transit Authority
55,Marin Transit


In [8]:
airtable.loc[airtable.issue_type.str.contains('Completeness')]

,gtfs_datasets,services,issue_type,description,airtable_ticket
9,Bay Area 511 Union City Transit VehiclePositions,Union City Transit,GTFS Realtime Completeness Problem,"In the past 14 days, GTFS-RT Vehicle Positions is 37.21%, which is a reduction from 46.15% VP recorded on August 25, 2023. Also, September GTFS-RT Vehicle Positions activity has fluctuated from 31.41% to 41.67%.",Yes
28,Lawndale Beat Vehicle Positions,Lawndale Beat,GTFS Realtime Completeness Problem,"In the past 14 days, GTFS-RT Vehicle Positions is 29.38%, which is a reduction from 33.33% VP recorded on August 25, 2023. Also, September GTFS-RT Vehicle Positions activity has fluctuated from 0% to 50%.",Yes
30,"MVGO VehiclePositions Historic,MVGO TripUpdates Historic",Mountain View Transportation Management Association,GTFS Realtime Completeness Problem,Since July 3 there have been zero trips accounted for in both the Vehicle Positions and Trip Updates for MVGO,Yes
32,Merced VehiclePositions,Merced The Bus,GTFS Realtime Completeness Problem,Transit Joint Powers Authority for Merced County: GTFS-RT issue: complete outage of GTFS-RT Vehicle Positions Data,Yes
40,PresidiGo VehiclePositions,Presidio GO Shuttle,GTFS Realtime Completeness Problem,Has dipped below 40% threshold of accounting for scheduled trips in the last 14 days as of 8/22.,Yes
42,"SLORTA Schedule,SLORTA VehiclePositions",SLO Transit,GTFS Realtime Completeness Problem,As of 929: only 15.74% of trips have trip updates and 15.6% of trips have VP messages.,Yes
43,San Diego Vehicle Positions,San Diego Metropolitan Transit System,GTFS Realtime Completeness Problem,"The system appears incapable of handling enough requests. Every hour there are at a couple of requests that return HTTP 500 responses which indicate a failure on the San Diego servers. In the past we tried downloading trip updates, but experienced worsening quality on the vehicle positions. We do not currently download trip update data because of this.",Yes
45,"Santa Rosa CityBus Schedule,Santa Rosa CityBus Vehicle Positions",Santa Rosa CityBus,GTFS Realtime Completeness Problem,As of 9/29: Only 4.9% of trips have trip update messages.,Yes
47,"SunLine Vehicle Positions,SunLine Trip Updates",SunLine Transit,GTFS Realtime Completeness Problem,"Since May 4, Sunline's Trip Updates and Vehicle Positions have consistently accounted for only 45% of scheduled trips.\n\nAs of Sept",Yes
49,Union City GMV Schedule,Union City Transit,GTFS Realtime Completeness Problem,"Per the Incompleteness dashboard, only 36.5% of trips have TU messages and 28.2% of trips have VP messages.",Yes


In [9]:
script.incomplete(
    "./gtfs_rt_trip_updates_completeness__last_14_days__2023-10-02T14_53_50.119519Z.xlsx",
    "./gtfs_rt_vehicle_positions_completeness__last_14_days__2023-10-02T14_53_53.891213Z.xlsx",
    airtable,
)

/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,name,%_of_trips_with_tu_messages,%_of_trips_with_vp_messages,gtfs_datasets,services,issue_type,description,airtable_ticket
0,Anaheim Resort Schedule,20.82,17.59,Anaheim Resort Schedule,Anaheim Resort Transportation,Service Accuracy,Anaheim Transportation Network: GTFS-Schedule issue: thousands of duplicate trips on Toy Story Line,Yes
1,Bay Area 511 ACE Schedule,0.00,0.00,NA,NA,NA,NA,NA
2,Bay Area 511 Angel Island-Tiburon Ferry Schedule,0.00,0.00,NA,NA,NA,NA,NA
3,Bay Area 511 BART Schedule,OK,0.00,NA,NA,NA,NA,NA
4,Bay Area 511 Capitol Corridor Schedule,0.00,0.00,NA,NA,NA,NA,NA
5,Bay Area 511 Commute.org Schedule,0.00,0.00,NA,NA,NA,NA,NA
6,Bay Area 511 Golden Gate Ferry Schedule,0.00,0.00,NA,NA,NA,NA,NA
7,Bay Area 511 MVGO Schedule,0.00,0.00,NA,NA,NA,NA,NA
8,Bay Area 511 Mission Bay Schedule,0.00,0.00,NA,NA,NA,NA,NA
9,Bay Area 511 Rio Vista Delta Breeze Schedule,0.00,0.00,NA,NA,NA,NA,NA
